In [19]:
import os
from dotenv import load_dotenv
from sqlalchemy.util import await_only

load_dotenv()
from langchain_mcp_adapters.client import MultiServerMCPClient

async def create_mcp_client():
    amap_key = os.environ.get("AMAP_KEY")

    client = MultiServerMCPClient({
        "amap": {
            "url": f"https://mcp.amap.com/sse?key={amap_key}",
            "transport": "sse",
        }
    })

    tools = await client.get_tools()

    return client, tools


In [31]:
# from pkg.tools.amap import create_mcp_client
from pkg.chat_model.qwen import chat_model

from langchain.agents import initialize_agent,AgentType
from langchain_core.prompts import PromptTemplate

client, tools = await create_mcp_client()

agent = initialize_agent(
    tools=tools,
    llm=chat_model,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

prompt_template = PromptTemplate.from_template(
    "你是一个智能助手，可以调用高德 MCP 工具。\n\n问题: {input}"
)

prompt = prompt_template.format(input="""
- 我五月底端午节计划去杭州游玩4天。
- 帮制作旅行攻略，考虑出行时间和路线，以及天气状况路线规划。
- 制作网页地图自定义绘制旅游路线和位置。
    - 网页使用简约美观页面风格，景区图片以卡片展示。
- 行程规划结果在高德地图app展示，并集成到h5页面中。
- 同一天行程景区之间我想打车前往。
""")
resp = await agent.ainvoke(prompt)
print(resp)



> Entering new AgentExecutor chain...
Thought: 我需要先查询杭州五月底端午节期间的天气情况，以便为旅行攻略提供参考。
Action:
```json
{
  "action": "maps_weather",
  "action_input": {
    "city": "杭州"
  }
}
```
Observation: {"city":"杭州市","forecasts":[{"date":"2025-11-05","week":"3","dayweather":"多云","nightweather":"晴","daytemp":"19","nighttemp":"14","daywind":"东","nightwind":"东","daypower":"1-3","nightpower":"1-3","daytemp_float":"19.0","nighttemp_float":"14.0"},{"date":"2025-11-06","week":"4","dayweather":"小雨","nightweather":"小雨","daytemp":"20","nighttemp":"15","daywind":"东","nightwind":"东","daypower":"1-3","nightpower":"1-3","daytemp_float":"20.0","nighttemp_float":"15.0"},{"date":"2025-11-07","week":"5","dayweather":"小雨","nightweather":"多云","daytemp":"21","nighttemp":"17","daywind":"东","nightwind":"东","daypower":"1-3","nightpower":"1-3","daytemp_float":"21.0","nighttemp_float":"17.0"},{"date":"2025-11-08","week":"6","dayweather":"小雨","nightweather":"小雨","daytemp":"22","nighttemp":"16","daywind":"西北","nightwind":"西北"